## IMPLEMENTACIÓN

In [3]:
from collections import deque
import time

class Nodo:
    def __init__(self, id_vertice):
        self.id = id_vertice
        self.vecinos = []  # [id_vecinos]
        self.distancia = float('inf') # numero nodos recorrido hasta el nodo actual
        self.padre = None
        self.visitado = False

    def agregar_vecino(self, vecino):
        self.vecinos.append(vecino.id)

    def __str__(self):
        return f'Nodo {self.id} conectado a: {[x for x in self.vecinos]}'

class Grafo:
    
    def __init__(self):
        self.vertices = {} # {id_vertice: objeto_Nodo}
        self.num_vertices = 0

    def agregar_vertice(self, id_vertice):
        self.num_vertices += 1
        nuevo_vertice = Nodo(id_vertice)
        self.vertices[id_vertice] = nuevo_vertice
        return nuevo_vertice

    def agregar_arista(self, id_origen, id_destino):
        if id_origen not in self.vertices:
            self.agregar_vertice(id_origen)
        if id_destino not in self.vertices:
            self.agregar_vertice(id_destino)

        self.vertices[id_origen].agregar_vecino(self.vertices[id_destino])
        self.vertices[id_destino].agregar_vecino(self.vertices[id_origen])

    # Algoritmo BFS
    def bfs(self, id_inicio, id_meta=None):
        """
        Ejecuta BFS y devuelve distancias, padres y la ruta corta.
        """
        if id_inicio not in self.vertices:
            return "Inicio no encontrado", None, None, 0

        start_time = time.time()
        search_queue = deque()
        
        # 1. Inicialización de estructuras
        for v in self.vertices.values():
            v.distancia = float('inf')
            v.padre = None
            v.visitado = False

        nodo_inicio = self.vertices[id_inicio]
        nodo_inicio.distancia = 0
        nodo_inicio.visitado = True
        search_queue += [nodo_inicio]
        
        # Estructuras de retorno (padres y distancias)
        padres = {id_inicio: None}
        distancias = {id_inicio: 0}
        
        while search_queue:
            nodo = search_queue.popleft()
            
            # Recorrer vecinos
            for id_vecino in nodo.vecinos:
                vecino = self.vertices[id_vecino]
                if not vecino.visitado:
                    vecino.visitado = True
                    vecino.distancia = nodo.distancia + 1
                    vecino.padre = nodo
                    
                    padres[vecino.id] = nodo.id
                    distancias[vecino.id] = vecino.distancia
                    search_queue.append(vecino)
                    
                    if id_meta and vecino.id == id_meta:
                        end_time = time.time()
                        return distancias, padres, self._reconstruir_ruta(id_inicio, id_meta, padres), end_time - start_time
        
        end_time = time.time()
        return distancias, padres, None, end_time - start_time
    
    # --- IMPLEMENTACIÓN DFS ---
    def dfs(self, id_inicio, id_meta=None):
        """
        Ejecuta DFS y devuelve padres, árbol, y la ruta encontrada.
        Algoritmo recursivo
        """
        if id_inicio not in self.vertices:
            return "Inicio no encontrado", None, None, None, 0

        start_time = time.time()

        # Inicializa los valores de los nodos
        for v in self.vertices.values():
            v.visitado = False
            v.padre = None
            
        padres = {id_inicio: None}
        ruta_encontrada = None
        
        def dfs_visita(u):
            nonlocal ruta_encontrada
            u.visitado = True
            
            # Explorar vecinos
            for id_vecino in u.vecinos:
                v = self.vertices[id_vecino]
                if not v.visitado:
                    v.padre = u
                    padres[v.id] = u.id
                    
                    if id_meta and v.id == id_meta and ruta_encontrada is None:
                        # Solo capturamos la primera ruta encontrada por DFS
                        ruta_encontrada = self._reconstruir_ruta(id_inicio, id_meta, padres)
                        
                    dfs_visita(v)
                    
        dfs_visita(self.vertices[id_inicio])

        # 2. Cálculo del camino de mayor profundidad (post-recorrido)
        camino_profundo, profundidad = self._encontrar_camino_mas_profundo(id_inicio, padres)
        
        end_time = time.time()
        return padres, ruta_encontrada, camino_profundo, end_time - start_time

    # --- MÉTODO AUXILIAR (Compartido por BFS/DFS) ---
    def _reconstruir_ruta(self, id_inicio, id_meta, padres):
        """Reconstruye la ruta desde la meta hasta el inicio usando el diccionario de padres."""
        ruta = []
        actual = id_meta
        while actual:
            ruta.append(actual)
            actual = padres.get(actual)
            if actual == id_inicio:
                ruta.append(id_inicio)
                break
        
        # Si la meta fue alcanzada, la ruta se construye al revés
        if ruta and ruta[-1] == id_inicio:
            return " -> ".join(ruta[::-1])
        return None

     # --- MÉTODO AUXILIAR: Encontrar Camino de Mayor Profundidad (para DFS) ---
    def _encontrar_camino_mas_profundo(self, id_inicio, padres):
        """
        Encuentra el camino con más nodos desde el inicio, 
        basándose en el árbol de padres de DFS.
        """
        nodo_mas_profundo = id_inicio
        max_profundidad = 0
        
        for nodo_id in self.vertices:
            # Calcular profundidad del nodo actual
            profundidad_actual = 0
            actual = nodo_id
            
            # Recorrer hacia el padre hasta llegar al inicio (None)
            temp_padre = padres.get(actual)
            while temp_padre:
                profundidad_actual += 1
                actual = temp_padre
                temp_padre = padres.get(actual)
            
            # Si el nodo no es alcanzable desde el inicio, su profundidad será 0.
            # Solo consideramos nodos que son descendientes del inicio.
            if actual == id_inicio and profundidad_actual > max_profundidad:
                max_profundidad = profundidad_actual
                nodo_mas_profundo = nodo_id
        
        # Reconstruir la ruta desde el nodo más profundo hasta el inicio
        if nodo_mas_profundo:
            return self._reconstruir_ruta(id_inicio, nodo_mas_profundo, padres), max_profundidad
        return None, 0

# Ejemplo de Ejecución
if __name__ == '__main__':
    
    g = Grafo()
    
    g.agregar_arista('A101', 'Pasillo_A') #origen, destino
    g.agregar_arista('Pasillo_A', 'B205')
    g.agregar_arista('Pasillo_A', 'Salida_E1')
    g.agregar_arista('Pasillo_A', 'Patio_Central')
    g.agregar_arista('Patio_Central', 'Biblioteca')
    g.agregar_arista('Patio_Central', 'Salida_E2')
    g.agregar_arista('Biblioteca', 'Salida_E2')
    g.agregar_arista('B205', 'Laboratorio')
    
    print("--- PRUEBA BFS (Ruta más corta / Evacuación Rápida) ---")
    inicio_bfs = 'A101'
    meta_bfs = 'Salida_E2'
    distancias, padres, ruta, tiempo = g.bfs(inicio_bfs, meta_bfs)
    
    if ruta:
        print(f"Ruta más corta de {inicio_bfs} a {meta_bfs}: {ruta}")
        print(f"Distancia (Número de nodos): {distancias.get(meta_bfs)}")
        print(f"Árbol BFS (Padres): {padres}")
        print(f"Tiempo de ejecución: {tiempo:.6f} segundos")
    else:
        print(f"No se encontró ruta de {inicio_bfs} a {meta_bfs}")

    print("\n--- PRUEBA DFS (Exploración / Verificación de Cobertura) ---")
    inicio_dfs = 'A101'
    meta_dfs_opcional = 'Laboratorio'
    padres_dfs, ruta_dfs_encontrada, camino_profundo, tiempo_dfs = g.dfs(inicio_dfs, meta_dfs_opcional)
    
    print(f"Árbol DFS (Padres de exploración): {padres_dfs}")
    print(f"Ruta DFS a la Meta '{meta_dfs_opcional}' (si existe): {ruta_dfs_encontrada}")
    print(f"Camino de Mayor Profundidad desde {inicio_dfs}: {camino_profundo}")
    print(f"Tiempo de ejecución: {tiempo_dfs:.6f} segundos")

--- PRUEBA BFS (Ruta más corta / Evacuación Rápida) ---
Ruta más corta de A101 a Salida_E2: A101 -> Pasillo_A -> Patio_Central -> Salida_E2
Distancia (Número de nodos): 3
Árbol BFS (Padres): {'A101': None, 'Pasillo_A': 'A101', 'B205': 'Pasillo_A', 'Salida_E1': 'Pasillo_A', 'Patio_Central': 'Pasillo_A', 'Laboratorio': 'B205', 'Biblioteca': 'Patio_Central', 'Salida_E2': 'Patio_Central'}
Tiempo de ejecución: 0.000061 segundos

--- PRUEBA DFS (Exploración / Verificación de Cobertura) ---
Árbol DFS (Padres de exploración): {'A101': None, 'Pasillo_A': 'A101', 'B205': 'Pasillo_A', 'Laboratorio': 'B205', 'Salida_E1': 'Pasillo_A', 'Patio_Central': 'Pasillo_A', 'Biblioteca': 'Patio_Central', 'Salida_E2': 'Biblioteca'}
Ruta DFS a la Meta 'Laboratorio' (si existe): A101 -> Pasillo_A -> B205 -> Laboratorio
Camino de Mayor Profundidad desde A101: A101 -> Pasillo_A -> Patio_Central -> Biblioteca -> Salida_E2
Tiempo de ejecución: 0.000051 segundos
